In [2]:
import math
import numpy as np
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model

In [3]:
import pandas as pd

We'll read the tweet file


In [4]:
clim = pd.read_csv('Climate_twitter.csv')
bt = pd.read_csv('twitter_sentiment_data.csv')
json = pd.read_json('train.jsonl', lines=True)

In [5]:
# adding label based on polarity score
conditions = [
    (clim['polarity'] < 0),
    (clim['polarity'] > 0),
    (clim['polarity'] == 0)]

values = [0,1, 'neutral']

clim['label'] = np.select(conditions, values)

clim.head()

,id,date,retweets,source,author,likes,text,twitter_name,location,verified,followers,friends,polarity,subjectivity,label
0,2184934963,2020-12-22 23:22:20,71,Twitter Web App,GO GREEN,91,The death of summer Arctic ice our Earth coole...,ECOWARRIORSS,NaN,False,23415,20439,-0.054365,0.426984,0
1,508658626,2020-12-10 14:30:00,14,Twitter for Advertisers,Elsevier Energy,98,Elsevier and the EditorsinChief are pleased to...,ElsevierEnergy,"Oxford, England",False,6615,508,0.387500,0.633333,1
2,2607105006,2020-12-22 21:28:52,0,Twitter Web App,Arwyn Thomas,1,From better climate change education to improv...,siwarr5,Carmarthen,False,22,133,0.261905,0.345238,1
3,19609660,2020-12-22 21:24:10,0,Twitter Web App,"Tom Gillispie, EDITOR/WRITER",0,climate change Links to FIXING CLIMATE CHANGE ...,EDITORatWORK,"Rural Hall, North Carolina, USA",False,4191,3708,0.000000,0.000000,neutral
4,19609660,2020-12-21 22:52:09,1,Twitter Web App,"Tom Gillispie, EDITOR/WRITER",1,climate change The 11TH HOUR FOR THE EARTH cli...,EDITORatWORK,"Rural Hall, North Carolina, USA",False,4191,3708,0.000000,0.000000,neutral


In [6]:
# dropping neutral entries
clim = clim.drop(clim[clim['label'] == 'neutral'].index)

In [7]:
# dropping all columns except text and label
clim = clim.drop(columns = ['id','date','source','author','twitter_name','location','verified','retweets','likes','followers','friends','polarity','subjectivity'])

In [8]:
clim.info()
clim['text'] = clim['text'].astype('string')
clim['label'] = clim['label'].astype('int64')
clim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 395
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    270 non-null    object
 1   label   270 non-null    object
dtypes: object(2)
memory usage: 6.3+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 395
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    270 non-null    string
 1   label   270 non-null    int64 
dtypes: int64(1), string(1)
memory usage: 6.3 KB


In [9]:
bt.head()
bt = bt.drop(columns=['tweetid'])

In [10]:
bt.rename(columns={'sentiment': "label", 'message':'text'}, inplace=True)

In [11]:
bt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43943 entries, 0 to 43942
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   43943 non-null  int64 
 1   text    43943 non-null  object
dtypes: int64(1), object(1)
memory usage: 686.7+ KB


In [12]:
bt['label'].value_counts() # imbalance

 1    22962
 2     9276
 0     7715
-1     3990
Name: label, dtype: int64

In [13]:
bt = bt.drop(bt[bt['label'] ==  2].index)

In [14]:
bt = bt.drop(bt[bt['label'] == 0].index)

In [15]:
bt.dropna(subset=['label'], inplace=True)

In [16]:
conditions = [
    (bt['label'] == -1),
    (bt['label'] == 1)]

values = [0,1]

bt['sent'] = np.select(conditions, values)

In [17]:
bt['label'] = bt['sent']
bt=bt.drop(columns=['sent'])

In [18]:
bt['text'] = bt['text'].astype("string")
bt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26952 entries, 0 to 43942
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   26952 non-null  int32 
 1   text    26952 non-null  string
dtypes: int32(1), string(1)
memory usage: 526.4 KB


In [19]:
bt['label'].value_counts()

1    22962
0     3990
Name: label, dtype: int64

In [20]:
json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2117 entries, 0 to 2116
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2117 non-null   object
 1   label   2117 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.2+ KB


In [21]:
json['label'].value_counts()

0    1585
1     532
Name: label, dtype: int64

In [22]:
df_list = [clim, bt, json]
df = pd.concat(df_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29339 entries, 0 to 2116
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    29339 non-null  object
 1   label   29339 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 687.6+ KB


In [23]:
df['text'] = df['text'].astype('string')

In [24]:
df['label'].value_counts()

1    23684
0     5655
Name: label, dtype: int64

In [25]:
import nltk

# Download the lexicon
nltk.download("vader_lexicon")

# Import the lexicon
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create an instance of SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sayan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [26]:
nltk.download('punkt')  # Download the punkt tokenizer if not already downloaded

df['tokens'] = df['text'].apply(nltk.word_tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sayan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
keywords = ["sustainability", "environmental", "conservation", "recycling", "sustainable",
                         "responsibility", "green", "eco-friendly", "renewable", "carbon", "climate", "ecosystem",
                         "planet", "biodiversity", "energy", "water", "pollution", "reduction", "renewability",
                         "ecological", "renewable", "greenhouse", "clean", "solar", "wind", "earth", "sustainable",
                         "planet", "ecology", "ocean", "forest", "organic", "earth-friendly", "bio", "ethics",
                         "conservationist", "sustain", "renew", "ethical", "greenery", "saver", "sustainable",
                         "conservator", "recycler", "biodegradable", "natural", "greenery", "environment",
                         "saver", "earth-saving", "sustainability", "green-living", "clean", "responsible",
                         "preservation", "regeneration", "ecosystem", "safeguarding"]
filtered_df = df[df['tokens'].apply(lambda tokens: any(keyword in tokens for keyword in keywords))]

In [28]:
filtered_df['label'].value_counts()

1    19287
0     2412
Name: label, dtype: int64

In [29]:
from sklearn.utils import resample
df1 = filtered_df.loc[filtered_df['label'] == 0]
df2 = filtered_df.loc[filtered_df['label'] == 1]
df1_sampled=resample(df1, replace=True,
                     n_samples= 19287,
                     random_state=42)
# reproducible results
filtered_df= pd.concat([df1_sampled, df2])
filtered_df.info()
filtered_df['label'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38574 entries, 20004 to 2115
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    38574 non-null  string
 1   label   38574 non-null  int64 
 2   tokens  38574 non-null  object
dtypes: int64(1), object(1), string(1)
memory usage: 1.2+ MB


0    19287
1    19287
Name: label, dtype: int64

In [30]:
label_to_sentiment = {0:"Negative", 1:"Positive"}
def mapper(label):
     return label_to_sentiment[label]
df.label = df.label.apply(lambda x: mapper(x))

In [31]:
# Import functions from sklearn library
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Splitting the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2,random_state=16)
print("Train Data size:", len(train_data))
print("Test Data size", len(test_data))
#> Train Data size: 1280000
#> Test Data size 320000

Train Data size: 23471
Test Data size 5868


In [32]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

In [33]:
tokenizer.fit_on_texts(train_data.text)
word_index = tokenizer.word_index
print(word_index)

{'climate': 1, 'change': 2, 'the': 3, 'rt': 4, 't': 5, 'https': 6, 'to': 7, 'co': 8, 'of': 9, 'is': 10, 'a': 11, 'and': 12, 'in': 13, 'global': 14, 'warming': 15, 'on': 16, 'we': 17, 'for': 18, 'that': 19, 'about': 20, 'are': 21, 'you': 22, 'it': 23, 'this': 24, 'our': 25, 'by': 26, 'i': 27, 'not': 28, 'how': 29, 'trump': 30, 'be': 31, 'believe': 32, 'have': 33, 'with': 34, 'will': 35, 'amp': 36, 'as': 37, 'q': 38, 'from': 39, 'but': 40, '…': 41, 'who': 42, 'at': 43, 'if': 44, 'real': 45, 'can': 46, 'your': 47, 'people': 48, 'more': 49, "doesn't": 50, 'all': 51, 'going': 52, 'so': 53, 'what': 54, 'world': 55, 'just': 56, 'has': 57, 'us': 58, "it's": 59, 'because': 60, 'do': 61, 'an': 62, 'now': 63, "she's": 64, 'no': 65, 'new': 66, 'like': 67, 'was': 68, 'when': 69, 'science': 70, 's': 71, 'fight': 72, 'they': 73, 'why': 74, 'president': 75, 'or': 76, 'up': 77, 'think': 78, "don't": 79, 'one': 80, 'out': 81, 'energy': 82, 'http': 83, 'he': 84, 'there': 85, 'need': 86, 'via': 87, 'die':

In [34]:
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)
#> Vocabulary Size : 290415

Vocabulary Size : 47460


In [35]:
from keras.preprocessing.sequence import pad_sequences
# The tokens are converted into sequences and then passed to the pad_sequences() function
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.text),maxlen = 30)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.text),maxlen = 30)

In [36]:
labels = ['Negative', 'Positive']
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train_data.label.to_list())
y_train = encoder.transform(train_data.label.to_list())
y_test = encoder.transform(test_data.label.to_list())
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [37]:
embeddings_index = {}
# opening the downloaded glove embeddings file
f = open("glove.6B.300d.txt", encoding='utf8')
for line in f:
    # For each line file, the words are split and stored in a list
    values = line.split()
    word = value = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [38]:
# creating an matrix with zeroes of shape vocab x embedding dimension
embedding_matrix = np.zeros((vocab_size, 300))
# Iterate through word, index in the dictionary
for word, i in word_index.items():
    # extract the corresponding vector for the vocab indice of same word
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Storing it in a matrix
        embedding_matrix[i] = embedding_vector

In [39]:
embedding_layer = tf.keras.layers.Embedding(vocab_size,300,weights=[embedding_matrix],
                                          input_length=30,trainable=False)

In [40]:
# Import various layers needed for the architecture from keras
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint
# The Input layer 
sequence_input = Input(shape=(30,), dtype='int32')
# Inputs passed to the embedding layer
embedding_sequences = embedding_layer(sequence_input)
# dropout and conv layer 
x = SpatialDropout1D(0.2)(embedding_sequences)
x = Conv1D(64, 5, activation='relu')(x)
x = Conv1D(64, 5, activation='relu')(x)
# Passed on to the LSTM layer
x = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
# Passed on to activation layer to get final output
outputs = Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(sequence_input, outputs)

In [41]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
model.compile(optimizer=Adam(), loss='binary_crossentropy',metrics=['accuracy'])
ReduceLROnPlateau = ReduceLROnPlateau(factor=0.1,min_lr = 0.01, monitor = 'val_loss',verbose = 1)

In [42]:
training = model.fit(x_train, y_train, batch_size=32, epochs=10,
                    validation_data=(x_test, y_test), callbacks=[ReduceLROnPlateau])

Epoch 1/10
734/734 [==============================] - 22s 25ms/step - loss: 0.3715 - accuracy: 0.8470 - val_loss: 0.3247 - val_accuracy: 0.8747 - lr: 0.0010
Epoch 2/10
734/734 [==============================] - 18s 24ms/step - loss: 0.2976 - accuracy: 0.8801 - val_loss: 0.2989 - val_accuracy: 0.8746 - lr: 0.0010
Epoch 3/10
734/734 [==============================] - 18s 24ms/step - loss: 0.2568 - accuracy: 0.8989 - val_loss: 0.2578 - val_accuracy: 0.8966 - lr: 0.0010
Epoch 4/10
734/734 [==============================] - 18s 24ms/step - loss: 0.2210 - accuracy: 0.9134 - val_loss: 0.2548 - val_accuracy: 0.8950 - lr: 0.0010
Epoch 5/10
734/734 [==============================] - 18s 24ms/step - loss: 0.1918 - accuracy: 0.9261 - val_loss: 0.2683 - val_accuracy: 0.8979 - lr: 0.0010
Epoch 6/10
734/734 [==============================] - 18s 24ms/step - loss: 0.1594 - accuracy: 0.9383 - val_loss: 0.2872 - val_accuracy: 0.8993 - lr: 0.0010
Epoch 7/10
734/734 [==============================] - 18s 

In [43]:
def predict_tweet_sentiment(score):
    return "Positive" if score>0.5 else "Negative"
scores = model.predict(x_test, verbose=1, batch_size=10000)
model_predictions = [predict_tweet_sentiment(score) for score in scores]

1/1 [==============================] - 1s 696ms/step


In [44]:
from sklearn.metrics import classification_report
print(classification_report(list(test_data.label), model_predictions))

              precision    recall  f1-score   support

    Negative       0.81      0.55      0.66      1103
    Positive       0.90      0.97      0.94      4765

    accuracy                           0.89      5868
   macro avg       0.86      0.76      0.80      5868
weighted avg       0.89      0.89      0.88      5868

